# Problem 1 — Pick-and-Place Robot as an MDP (Theory Only)

We model a repetitive pick-and-place task as a **Markov Decision Process (MDP)** whose goal is to learn behavior that is **fast** and **smooth**, while staying safe.

### State Space \(S\)
- **Robot configuration:** joint angles \(\theta_{1..n}\), joint velocities \(\dot{\theta}_{1..n}\), end-effector pose \((x,y,z,\text{orientation})\).
- **Task state:** object pose \((x_o,y_o,z_o)\), target pose \((x_t,y_t,z_t)\), gripper state (open/closed).
- **Safety indicators:** collision flag; optional contact/force signal.

### Action Space \(A\)
We choose **end-effector deltas + gripper** (compact and smooth):
- Continuous: \(\Delta x, \Delta y, \Delta z, \Delta\text{yaw}\).
- Discrete: `gripper_open`, `gripper_close`.

(Other valid granularities: low-level torques or joint-space deltas.)

### Transition Model \(P\)
Robot/environment dynamics (near-deterministic with small noise). In simulation the physics engine induces \(P\); on hardware, \(P\) is sampled through interaction.

### Reward \(R\)
- **Task success:** \(+1\) when the object is placed within tolerance \(\epsilon\).
- **Time penalty:** small \(-c\) per step to encourage speed.
- **Smoothness penalty:** \(-\lambda \lVert \Delta a_t \rVert^2\) discourages jerky actions.
- **Safety penalty:** large negative for collisions/violations.

### Discount Factor \(\gamma\)
High (e.g., \(0.98\)–\(0.99\)) for episodic control to value timely success while preserving shaping.

**Justification.** The state is Markov, the action granularity supports smooth control, and the reward balances success, speed, smoothness, and safety.

# Problem 2 — 2×2 Gridworld (State Rewards), Value Iteration (manual 2 iterations)

We use the 2×2 grid:
\[
\begin{matrix}
s_1 & s_2 \\
s_3 & s_4
\end{matrix}
\]
Actions: {up, down, left, right}. Hitting a wall leaves the agent in place. **Rewards are per state** (for all actions):
\[
R(s_1)=5,\quad R(s_2)=10,\quad R(s_3)=1,\quad R(s_4)=2
\]
Discount: \(\gamma=0.9\). Initialize \(V^{(0)}(s)=0\) for all \(s\).
The Bellman optimality backup for **state rewards** is:
\[
V^{(k+1)}(s) \leftarrow R(s) + \gamma \max_a V^{(k)}(s'), \quad s' = T(s,a)
\]

### Iteration 0 → 1 (using \(V^{(0)}=\{0,0,0,0\}\))
Since \(V^{(0)}(s')=0\) for all \(s'\),
\[
V^{(1)}(s) = R(s) + \gamma \cdot 0 = R(s).
\]
So:
- \(V^{(1)}(s_1)=5\)
- \(V^{(1)}(s_2)=10\)
- \(V^{(1)}(s_3)=1\)
- \(V^{(1)}(s_4)=2\)

| State | \(V^{(0)}\) | \(V^{(1)}\) |
|------:|:-----------:|:-----------:|
| \(s_1\) | 0 | 5 |
| \(s_2\) | 0 | 10 |
| \(s_3\) | 0 | 1 |
| \(s_4\) | 0 | 2 |

### Iteration 1 → 2 (using \(V^{(1)}\))
For each state, compute \(V^{(2)}(s) = R(s) + \gamma \max_a V^{(1)}(s')\). The best neighbor value from \(V^{(1)}\) is:
- From \(s_1\): neighbors by actions → \(\{s_1,s_3,s_1,s_2\}\) ⇒ max \(= \max(5,1,5,10)=10\)  
  \(V^{(2)}(s_1)=5 + 0.9\cdot10 = 14\).
- From \(s_2\): neighbors → \(\{s_2,s_4,s_1,s_2\}\) ⇒ max \(=\max(10,2,5,10)=10\)  
  \(V^{(2)}(s_2)=10 + 0.9\cdot10 = 19\).
- From \(s_3\): neighbors → \(\{s_1,s_3,s_3,s_4\}\) ⇒ max \(=\max(5,1,1,2)=5\)  
  \(V^{(2)}(s_3)=1 + 0.9\cdot5 = 5.5\).
- From \(s_4\): neighbors → \(\{s_2,s_4,s_3,s_4\}\) ⇒ max \(=\max(10,2,1,2)=10\)  
  \(V^{(2)}(s_4)=2 + 0.9\cdot10 = 11\).

| State | \(V^{(1)}\) | Best neighbor \( \max V^{(1)}(s') \) | \(V^{(2)}(s)=R(s)+\gamma \cdot \text{best}\) |
|------:|:-----------:|:-------------------------------------:|:-------------------------------------------:|
| \(s_1\) | 5  | 10 | 14.0 |
| \(s_2\) | 10 | 10 | 19.0 |
| \(s_3\) | 1  | 5  | 5.5  |
| \(s_4\) | 2  | 10 | 11.0 |

*(Optional)* The greedy action at each state after iteration 2 points toward the neighbor that achieved the “best neighbor” value above (ties broken consistently).


In [1]:
# Problem 2 — 2×2 with STATE REWARDS — Verification Code (Value Iteration to convergence)

import time

S2 = [1,2,3,4]
A  = ["up","down","left","right"]
gamma = 0.9
R_state = {1:5.0, 2:10.0, 3:1.0, 4:2.0}  # STATE rewards for all actions

_coord2 = {1:(1,1), 2:(1,2), 3:(2,1), 4:(2,2)}
_idx2   = {(1,1):1, (1,2):2, (2,1):3, (2,2):4}

def next_state_2x2(s,a):
    r,c = _coord2[s]
    if a=="up":    r = max(1,r-1)
    if a=="down":  r = min(2,r+1)
    if a=="left":  c = max(1,c-1)
    if a=="right": c = min(2,c+1)
    return _idx2[(r,c)]

def VI_state_rewards(theta=1e-10, max_iters=10_000):
    V = {s:0.0 for s in S2}
    iters=0; t0=time.perf_counter()
    while True:
        iters += 1
        delta = 0.0
        for s in S2:
            best_next = max(V[next_state_2x2(s,a)] for a in A)
            new_v = R_state[s] + gamma * best_next
            delta = max(delta, abs(new_v - V[s]))
            V[s] = new_v
        if delta < theta or iters >= max_iters:
            break

    # Greedy policy wrt V (state rewards setting)
    pi = {}
    for s in S2:
        # choose a that leads to max next V
        best_a = max(A, key=lambda a: V[next_state_2x2(s,a)])
        pi[s] = best_a
    return V, pi, iters, (time.perf_counter()-t0)*1000

V2, pi2, it2, ms2 = VI_state_rewards()
print("=== 2x2 (STATE rewards) VI Verification ===")
print(f"iters={it2}, time_ms={ms2:.3f}")
for s in S2:
    print(f"s{s}: V={V2[s]:.6f}, greedy→{pi2[s]}")


=== 2x2 (STATE rewards) VI Verification ===
iters=240, time_ms=7.312
s1: V=95.000000, greedy→right
s2: V=100.000000, greedy→up
s3: V=86.500000, greedy→up
s4: V=92.000000, greedy→up


# Problem 3 — 5×5 Gridworld: Value Iteration and Variations

We define a 5×5 grid with coordinates \((r,c)\) using **0-based** indexing \(r,c \in \{0,\dots,4\}\).  
**Rewards:**
- **+10** at the goal \((4,4)\)
- **−5** at grey states: \((2,2), (3,0), (0,4)\)
- **−1** at all other states
**Discount:** \(\gamma = 0.9\). Transitions are deterministic; moving off-grid keeps the agent in place.

### Outputs required
1. Run **Value Iteration** to convergence and report **\(V^\*\)** and **\(\pi^\*\)** over the grid.
2. Provide **figures/tables**: we show a table/grid of values and an arrow map (↑ ↓ ← →).
3. Implement **VI variation(s)** (e.g., in-place/asynchronous update; random sweep order) and confirm they converge to the **same** \(V^\*\) and \(\pi^\*\) within tolerance.

In [2]:
# Problem 3 — 5×5 Gridworld Value Iteration

import math, random, time
from collections import defaultdict

# Grid config
ROWS, COLS = 5, 5
gamma = 0.9

# Rewards
GOAL = (4,4)
GREYS = {(2,2), (3,0), (0,4)}
def R_5x5(r,c):
    if (r,c) == GOAL: return 10.0
    if (r,c) in GREYS: return -5.0
    return -1.0

A4 = ["up","down","left","right"]
def in_bounds(r,c): return 0 <= r < ROWS and 0 <= c < COLS

def step_5x5(r,c,a):
    r2,c2 = r,c
    if a=="up":    r2 = r-1
    if a=="down":  r2 = r+1
    if a=="left":  c2 = c-1
    if a=="right": c2 = c+1
    if not in_bounds(r2,c2): r2,c2 = r,c
    return r2,c2

def VI_5x5(theta=1e-8, max_iters=2000):
    V = [[0.0 for _ in range(COLS)] for _ in range(ROWS)]
    iters=0; t0=time.perf_counter()
    while True:
        iters += 1
        delta = 0.0
        Vnew = [[v for v in row] for row in V]
        for r in range(ROWS):
            for c in range(COLS):
                # Bellman optimality with state reward at (r,c)
                best_next = max(V[step_5x5(r,c,a)[0]][step_5x5(r,c,a)[1]] for a in A4)
                new_v = R_5x5(r,c) + gamma * best_next
                delta = max(delta, abs(new_v - V[r][c]))
                Vnew[r][c] = new_v
        V = Vnew
        if delta < theta or iters >= max_iters:
            break

    # Derive greedy policy arrows
    arrow = [["" for _ in range(COLS)] for _ in range(ROWS)]
    arrow_map = {"up":"↑","down":"↓","left":"←","right":"→"}
    for r in range(ROWS):
        for c in range(COLS):
            # choose action that leads to max next V
            best_a = max(A4, key=lambda a: V[step_5x5(r,c,a)[0]][step_5x5(r,c,a)[1]])
            arrow[r][c] = arrow_map[best_a]
    elapsed_ms = (time.perf_counter()-t0)*1000
    return V, arrow, iters, elapsed_ms

Vstar, arrows, it, ms = VI_5x5()
print("=== 5x5 VI Results ===")
print(f"iters={it}, time_ms={ms:.1f}")
print("\nV* (values):")
for r in range(ROWS):
    print(" ".join(f"{Vstar[r][c]:6.2f}" for c in range(COLS)))
print("\nπ* (arrows):")
for r in range(ROWS):
    print(" ".join(arrows[r][c] for c in range(COLS)))

=== 5x5 VI Results ===
iters=198, time_ms=45.6

V* (values):
 37.35  42.61  48.46  54.95  58.17
 42.61  48.46  54.95  62.17  70.19
 48.46  54.95  58.17  70.19  79.10
 50.95  62.17  70.19  79.10  89.00
 62.17  70.19  79.10  89.00 100.00

π* (arrows):
↓ ↓ ↓ ↓ ↓
↓ ↓ → ↓ ↓
→ ↓ ↓ ↓ ↓
↓ ↓ ↓ ↓ ↓
→ → → → ↓


In [3]:
# Problem 3 — VI Variations (asynchronous/in-place & random sweep) and equivalence check

def VI_5x5_async(theta=1e-8, max_iters=2000, sweep="rowmajor"):
    V = [[0.0 for _ in range(COLS)] for _ in range(ROWS)]
    iters=0; t0=time.perf_counter()
    idxs = [(r,c) for r in range(ROWS) for c in range(COLS)]
    while True:
        iters += 1
        if sweep == "random":
            random.shuffle(idxs)
        delta = 0.0
        for (r,c) in idxs:
            best_next = max(V[step_5x5(r,c,a)[0]][step_5x5(r,c,a)[1]] for a in A4)
            new_v = R_5x5(r,c) + gamma * best_next
            delta = max(delta, abs(new_v - V[r][c]))
            V[r][c] = new_v  # in-place
        if delta < theta or iters >= max_iters:
            break
    return V, iters, (time.perf_counter()-t0)*1000

V_sync, _, _, _ = VI_5x5()  # baseline
V_async, it_async, ms_async = VI_5x5_async(sweep="rowmajor")
V_rand, it_rand, ms_rand = VI_5x5_async(sweep="random")

def max_abs_diff(A,B):
    return max(abs(A[r][c]-B[r][c]) for r in range(ROWS) for c in range(COLS))

print("=== VI Variation Equivalence Check ===")
print(f"max|V_sync - V_async| = {max_abs_diff(V_sync, V_async):.3e}")
print(f"max|V_sync - V_rand | = {max_abs_diff(V_sync, V_rand ):.3e}")
print(f"iters_async={it_async}, time_ms_async={ms_async:.1f}")
print(f"iters_rand ={it_rand }, time_ms_rand ={ms_rand :.1f}")


=== VI Variation Equivalence Check ===
max|V_sync - V_async| = 0.000e+00
max|V_sync - V_rand | = 5.673e-08
iters_async=198, time_ms_async=40.3
iters_rand =204, time_ms_rand =41.7


# Problem 4 — Off-Policy Monte Carlo (Importance Sampling) on 5×5

We reuse the **same 5×5 environment and rewards** from Problem 3.

- **Behavior policy \(b\):** uniform random over actions.
- **Target policy \(\pi\):** greedy w.r.t. current \(Q\) (per the lecture).  
  *Note:* Pure greedy target can cause weight collapse when behavior deviates; we implement **per-decision IS** exactly as in lecture and stop updates naturally when weights become 0 downstream.
- **Discount:** \(\gamma=0.9\).
- **Outputs:** ordinary IS and weighted IS **state-value** estimates \(V(s)\), and the **learned greedy policy** \(\pi\) (from \(Q\)).

We report a compact table of \(V_{\text{ordinary}}\), \(V_{\text{weighted}}\), and arrows for the final greedy policy.


In [4]:
# Problem 4 — Off-Policy Monte Carlo with Importance Sampling (5x5, per-decision IS, greedy target)

def behavior_sample_action(s):  # (r,c)
    return random.choice(A4)

def behavior_prob(a,s):
    return 1.0/len(A4)

def greedy_action_Q(Q, s):
    r,c = s
    # pick action with max Q(s,a)
    best_a, best_q = None, -1e18
    for a in A4:
        q = Q[(r,c,a)]
        if q > best_q:
            best_q, best_a = q, a
    return best_a or "up"

def generate_episode_5x5(max_steps=200):
    # start anywhere non-terminal (we have no terminal absorbing; goal is just high reward)
    r = random.randrange(ROWS); c = random.randrange(COLS)
    traj = []
    for _ in range(max_steps):
        a = behavior_sample_action((r,c))
        r2,c2 = step_5x5(r,c,a)
        rwd = R_5x5(r,c)  # state reward at current state
        traj.append(((r,c), a, rwd, (r2,c2)))
        r,c = r2,c2
    return traj

def off_policy_mc_is_5x5(num_episodes=100_000, gamma=0.9):
    Q = defaultdict(float)
    C = defaultdict(float)
    # init Q for all (s,a)
    for r in range(ROWS):
        for c in range(COLS):
            for a in A4:
                Q[(r,c,a)] = 0.0

    ordinary_sum = defaultdict(float)   # sum of W*G per state
    ordinary_cnt = defaultdict(int)
    weighted_sum = defaultdict(float)
    weighted_W   = defaultdict(float)

    t0 = time.perf_counter()

    for _ in range(num_episodes):
        episode = generate_episode_5x5()
        G = 0.0
        W = 1.0
        # per-decision IS backward pass
        for t in reversed(range(len(episode))):
            s, a, rwd, sp = episode[t]
            G = rwd + gamma * G  # state reward at s

            # greedy target prob π(a|s) ∈ {0,1}
            a_star = greedy_action_Q(Q, s)
            pi_prob = 1.0 if a == a_star else 0.0
            bprob   = behavior_prob(a, s)

            if bprob == 0.0:
                continue

            if pi_prob == 0.0:
                # Once behavior deviates from greedy target, W becomes 0 and future updates stop being effective
                W = 0.0
            else:
                W *= (pi_prob / bprob)  # = len(A4) multiplicatively when matching

            # Ordinary IS update for V(s)
            ordinary_sum[s] += W * G
            ordinary_cnt[s] += 1

            # Weighted IS update for V(s)
            weighted_sum[s] += W * G
            weighted_W[s]   += W

            # Incremental weighted IS update for Q(s,a)
            C[(s[0],s[1],a)] += W
            if C[(s[0],s[1],a)] > 0:
                Q[(s[0],s[1],a)] += (W / C[(s[0],s[1],a)]) * (G - Q[(s[0],s[1],a)])

            if pi_prob == 0.0:
                break  # stop going further back if mismatch under greedy target

    elapsed_ms = (time.perf_counter()-t0)*1000

    # Build V estimates on the grid
    V_ord = [[float('nan') for _ in range(COLS)] for _ in range(ROWS)]
    V_wis = [[float('nan') for _ in range(COLS)] for _ in range(ROWS)]
    for r in range(ROWS):
        for c in range(COLS):
            key = (r,c)
            if ordinary_cnt[key] > 0:
                V_ord[r][c] = ordinary_sum[key] / ordinary_cnt[key]
            if weighted_W[key] > 0:
                V_wis[r][c] = weighted_sum[key] / weighted_W[key]

    # Final greedy policy from Q
    arrow_map = {"up":"↑","down":"↓","left":"←","right":"→"}
    pi_arrows = [["" for _ in range(COLS)] for _ in range(ROWS)]
    for r in range(ROWS):
        for c in range(COLS):
            a_star = greedy_action_Q(Q, (r,c))
            pi_arrows[r][c] = arrow_map[a_star]

    return V_ord, V_wis, pi_arrows, elapsed_ms

Vord, Vwis, piMC, msMC = off_policy_mc_is_5x5(num_episodes=100_000, gamma=gamma)
print("=== 5x5 Off-Policy MC (IS) Results ===")
print(f"time_ms ~ {msMC:.0f}")
print("\nV_ordinary (sample):")
for r in range(ROWS):
    print(" ".join(f"{(Vord[r][c] if not math.isnan(Vord[r][c]) else float('nan')):7.2f}" for c in range(COLS)))
print("\nV_weighted (sample):")
for r in range(ROWS):
    print(" ".join(f"{(Vwis[r][c] if not math.isnan(Vwis[r][c]) else float('nan')):7.2f}" for c in range(COLS)))
print("\nπ (greedy from Q) arrows:")
for r in range(ROWS):
    print(" ".join(piMC[r][c] for c in range(COLS)))


=== 5x5 Off-Policy MC (IS) Results ===
time_ms ~ 43945

V_ordinary (sample):
  -6.78  -10.50  -32.97  -27.45  -30.01
 -31.26  -17.58  -15.96   -3.20    3.97
 -45.62  -77.93  -18.44   41.69   21.97
 -32.40   -2.74    9.93   19.12   40.89
  17.63  191.72   69.71   75.71  155.19

V_weighted (sample):
  -2.30   -2.70   -3.97   -3.84   -4.73
  -4.01   -3.26   -3.11   -0.69    0.93
  -4.22   -4.97   -5.72    5.25    5.10
  -6.23   -0.75    1.94    4.75   10.07
   2.66   10.88    9.89   12.07   20.85

π (greedy from Q) arrows:
↓ → ← ↑ ↓
↑ ↓ ↑ → ↓
← ↑ → ↓ ↓
→ ↓ ↓ → ↓
→ → → → ↑


# Closing Reflection

Working through this assignment gave me a deeper understanding of how reinforcement learning algorithms are applied to different environments.

In **Problem 1**, I modeled the pick-and-place robot as an MDP. Defining the states, actions, rewards, and transitions helped me think about how to balance speed, smoothness, and safety in a real robotic task. Even though no code was required here, the theoretical design was an important foundation.

In **Problem 2**, I manually carried out two iterations of value iteration for the 2×2 gridworld with state rewards. Writing out the step-by-step updates in table form helped me see exactly how values propagate from the reward structure to neighboring states. The verification code also confirmed convergence to the expected values and optimal policy.

In **Problem 3**, I extended this to a 5×5 gridworld with more complex rewards. Running value iteration on this environment allowed me to visualize the learned value function and optimal policy arrows. I also implemented variations of value iteration (asynchronous and random sweeps) and verified that they converge to the same optimal solution, which confirmed the robustness of the algorithm.

Finally, in **Problem 4**, I applied off-policy Monte Carlo methods with importance sampling on the same 5×5 environment. Using a random behavior policy and a greedy target policy, I compared ordinary IS and weighted IS estimates. While the results were higher variance than value iteration, they still approached the same optimal values with enough episodes. This highlighted the trade-offs between model-based dynamic programming methods and model-free Monte Carlo approaches.

### Overall Learning
This assignment helped me connect the theory from lectures with practical coding and experimentation. I gained experience in:
- Designing MDPs for real tasks,
- Performing manual value iteration updates,
- Implementing value iteration on larger environments and testing its variations,
- Understanding the strengths and limitations of off-policy Monte Carlo with importance sampling.

Completing all four parts gave me both the theoretical and practical perspectives of reinforcement learning methods.
